In [1]:
#from Jon 

import pandas as pd
from bs4 import BeautifulSoup



In [12]:
# Read in the raw HTML file and load it into BeautifulSoup
with open('Event1.html') as inf:
    soup = BeautifulSoup(inf.read(), 'html.parser')


In [13]:

# Dump out the HTML in an easier to read format
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="width=device-width,initial-scale=1,maximum-scale=1,minimum-scale=1,user-scalable=no" name="viewport"/>
  <meta content="no-cache, no-store, must-revalidate" http-equiv="Cache-Control"/>
  <meta content="no-cache" http-equiv="Pragma"/>
  <meta content="0" http-equiv="Expires"/>
  <!-- -->
  <!-- Consider inlining CSS to reduce the number of requested files -->
  <!-- -->
  <link href="StyleSheetServlet" rel="stylesheet" type="text/css"/>
  <!-- -->
  <!-- Any title is fine                         -->
  <!-- -->
  <title>
   Event Entries
  </title>
  <!-- -->
  <!-- This script loads your compiled module.   -->
  <!-- If you add any GWT meta tags, they must   -->
  <!-- be added before this line.                -->
  <!-- -->
  <script src="https://www.googletagservices.com/activeview/js/current/osd.js?cb=%2Fr20100101">
  </script>
  <script src="https://partner.googlea

In [14]:
# Find all results tables. Each table is wrapped by a div with class "brd-div"
# so we can find all those divs and then access the child table within each div
for div in soup.find_all('div', class_="brd-div"):
    table = div.find_next('table')
    # Now we have the table:
    # - the first row is the name of the trial
    # - the 2nd row is the "Last updated" row
    # - the 3rd row is the column headers
    # - all remaining rows are the data

   

In [24]:
 # Get the first row
    trial_row = table.find_next('tr')
    # Find the a tag and retrieve the inner text - this is the trial (Level) name string
    trial = trial_row.find_next('a').contents[0]
    print(trial)
    

   

Starter


In [25]:
 # Get the next row (row 2)
    last_updated_row = trial_row.next_sibling
    # If you want the last updated time, pull it out here

    print(last_updated_row)

<tr><td colspan="15"><div><div class="gwt-HTML lupdated">Last Updated 01/12/2020 04:26 PM</div></div></td></tr>


In [17]:
# Get the next row (row 3)
    col_header_row = last_updated_row.next_sibling
    # Iterate through the columns (td elements) in the row to extract the
    # column headers
    col_headers = []
    for td in col_header_row.find_all('td'):
        # The td elements we want have an inner div element with a class value
        # of gwt-HTML, the contents of that div are the label we want and the
        # label is split across two lines except the Rider/Horse column which
        # we need to treat as 2 separate columns
        if len(td.contents) == 3:
            # This is the Rider/Horse td
            col_headers.append(td.contents[0])
            col_headers.append(td.contents[2])
        else:
            inner_div = td.find('div', class_='gwt-HTML')
            if inner_div:
                label = inner_div.contents[0] + " " + inner_div.contents[2]
                if label == 'Rider Horse':
                    col_headers += label.split(' ')
                else:
                    col_headers.append(label)

  

In [22]:
  # Now iterate over the remaining rows in the table which contain the data
    # Every row contains a div with class gwt-HTML like the header row except
    # the Rider/Horse column which has 2 values separated by a <br/> tag
    data_rows = []
    for row in col_header_row.next_siblings:
        data_row = []
        for td in row.find_all('td'):
            if len(td.contents) == 3:
                # This is the Rider/Horse td
                data_row.append(td.contents[0])
                # The Horse name is wrapped in <i> tags so we get the contents
                # of that inner tag
                data_row.append(td.contents[2].contents[0])
            else:
                inner_div = td.find('div', class_='gwt-HTML')
                if inner_div:
                    # The content of this div is the data value
                    data_row.append(inner_div.contents[0])
        data_rows.append(data_row)
    majesticoaksdf = pd.DataFrame.from_records(data_rows, columns=col_headers)
    print(majesticoaksdf)

                Rider                                Horse Dress Score  \
0       Jamie Bassett                       Master Higgins       29.40   
1       Kristen Ayers                          Pack Leader       32.50   
2       Diana Bregman                   Green Mountain Sky       33.30   
3    Ryleigh  Briles                  NDR's Charlie Horse        36.70   
4       Chantel Drago                                 Owen       36.40   
5  Jennifer Carpenter                           Miss Bizzi       34.20   
6        Susan Lester  Leprechaun Lane's Roxii's Got Moxie          TE   

  Dress Place Stad Jump Stad Time To Date Score To Date Place XC Jump  \
0           1      0.00      0.80         30.20             1    0.00   
1           2      0.00      0.00         32.50             2    0.00   
2           3      4.00      0.00         37.30             3    0.00   
3           6      8.00      0.00         44.70             4    0.00   
4           5      4.00      6.00         

In [23]:
majesticoaksdf.count()


Rider            7
Horse            7
Dress Score      7
Dress Place      7
Stad Jump        7
Stad Time        7
To Date Score    7
To Date Place    7
XC Jump          7
XC Elapsed       7
XC Time          7
Final Score      7
Final Place      7
dtype: int64

TypeError: 'tuple' object is not callable